In [1]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import numpy as np
import csv


Using TensorFlow backend.


In [2]:
batch_size = 64
epochs     = 40
latent_dim = 50
num_samples= 10000


data_path = "./headlines.csv"


input_headlines     = []
target_values       = []
input_characters    = set()


In [4]:
with open(data_path,  "r", encoding='utf-8') as f:
    csv_reader = csv.reader(f, delimiter=",")
    for row in csv_reader: 
        #print(row)
        row = [r[2:-1].upper() for r in row]
        input_headlines.append("\n".join(row[2:]))
        #print(row)
        target_values.append(row[1])

head_lengths = []
for head in input_headlines:
    head_lengths.append(len(head))
    for char in head:
        if char not in input_characters:
            input_characters.add(char)

input_characters    = sorted(list(input_characters))
num_encoder_tokens  = len(input_characters)

max_encoder_seq_length  = max(head_lengths)





NameError: name 'input_texts' is not defined

In [5]:
print('Number of samples:', len(input_headlines))
print('Number of unique input tokens:', num_encoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)

Number of samples: 1991
Number of unique input tokens: 68
Max sequence length for inputs: 4349


In [6]:
input_token_index  = dict([(char,i) for i, char in enumerate(input_characters)])

encoder_input_data = np.zeros((len(input_headlines), 25, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_headlines), 1), dtype="float32")
decoder_target_data = np.zeros((len(input_headlines), 1), dtype="float32")




MemoryError: 

In [133]:
for i, (headlines, value) in enumerate(zip(input_headlines, target_values)):
    for h,head in enumerate(headlines):
        for t,char in enumerate(head):
            encoder_input_data[i,h,t,input_token_index[char]] = 1.



In [134]:

encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation="softmax")
decoder_outputs= decoder_dense(decoder_outputs)

In [122]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss='categorical_crossentropy')
model.fit([encoder_input_data,decoder_input_data],decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save('heb2.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 139s 17ms/step - loss: 0.9379 - val_loss: 0.9937
Epoch 2/20
8000/8000 [==============================] - 140s 17ms/step - loss: 0.7841 - val_loss: 0.8721
Epoch 3/20
8000/8000 [==============================] - 140s 18ms/step - loss: 0.7061 - val_loss: 0.8245
Epoch 4/20
8000/8000 [==============================] - 95s 12ms/step - loss: 0.6605 - val_loss: 0.7801
Epoch 5/20
8000/8000 [==============================] - 86s 11ms/step - loss: 0.6225 - val_loss: 0.7488
Epoch 6/20
8000/8000 [==============================] - 87s 11ms/step - loss: 0.5923 - val_loss: 0.7212
Epoch 7/20
8000/8000 [==============================] - 87s 11ms/step - loss: 0.5680 - val_loss: 0.7030
Epoch 8/20
8000/8000 [==============================] - 87s 11ms/step - loss: 0.5469 - val_loss: 0.6917
Epoch 9/20
8000/8000 [==============================] - 87s 11ms/step - loss: 0.5290 - val_loss: 0.6705
Epoch 10/20
8

C:\Users\pedoe\Miniconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer lstm_16 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_15/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_15/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [135]:
model = load_model("heb2.h5")
model.compile(optimizer="rmsprop", loss='categorical_crossentropy')
model.fit([encoder_input_data,decoder_input_data],decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/40
8000/8000 [==============================] - 110s 14ms/step - loss: 0.3797 - val_loss: 0.6212
Epoch 2/40
8000/8000 [==============================] - 114s 14ms/step - loss: 0.3660 - val_loss: 0.6247
Epoch 3/40
8000/8000 [==============================] - 113s 14ms/step - loss: 0.3562 - val_loss: 0.6296
Epoch 4/40
8000/8000 [==============================] - 110s 14ms/step - loss: 0.3464 - val_loss: 0.6257
Epoch 5/40
8000/8000 [==============================] - 110s 14ms/step - loss: 0.3370 - val_loss: 0.6356
Epoch 6/40
8000/8000 [==============================] - 108s 14ms/step - loss: 0.3283 - val_loss: 0.6375
Epoch 7/40
8000/8000 [==============================] - 108s 13ms/step - loss: 0.3196 - val_loss: 0.6363
Epoch 8/40
8000/8000 [==============================] - 108s 14ms/step - loss: 0.3105 - val_loss: 0.6470
Epoch 9/40
8000/8000 [==============================] - 108s 14ms/step - loss: 0.3025 - val_loss: 0.6461
Epoch 1

In [127]:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_state_inputs)
decoder_states = [state_h, state_c]

decoder_outputs= decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i,char) for char, i  in input_token_index.items())
reverse_target_char_index = dict((i,char) for char, i  in target_token_index.items())

In [128]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq   = np.zeros((1,1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or  len(decoded_sentence)> max_decoder_seq_length):    
            stop_condition = True
        target_seq   = np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0,sampled_token_index] = 1.
        states_value = [h,c]

    return decoded_sentence


In [129]:

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("_")
    print("Input Sentence: ", input_texts[seq_index])
    print("Deocded Sentence: ", decoded_sentence) 
    #print("Deocded Sentence: ", decoded_sentence[::-1])    

_
Input Sentence:  Hi.
Deocded Sentence:  היה נחמד.

_
Input Sentence:  Hi.
Deocded Sentence:  היה נחמד.

_
Input Sentence:  Run!
Deocded Sentence:  רוצו!

_
Input Sentence:  Run!
Deocded Sentence:  רוצו!

_
Input Sentence:  Run!
Deocded Sentence:  רוצו!

_
Input Sentence:  Who?
Deocded Sentence:  מי האה?

_
Input Sentence:  Fire!
Deocded Sentence:  סדור את זה!

_
Input Sentence:  Fire!
Deocded Sentence:  סדור את זה!

_
Input Sentence:  Help!
Deocded Sentence:  תפסיק ליקות!

_
Input Sentence:  Jump.
Deocded Sentence:  אפוץ.

_
Input Sentence:  Stop!
Deocded Sentence:  תפסיק להקשק!

_
Input Sentence:  Go on.
Deocded Sentence:  תעשיו את זה.

_
Input Sentence:  Go on.
Deocded Sentence:  תעשיו את זה.

_
Input Sentence:  Go on.
Deocded Sentence:  תעשיו את זה.

_
Input Sentence:  Hello!
Deocded Sentence:  תפסיק ליקות!

_
Input Sentence:  Hello!
Deocded Sentence:  תפסיק ליקות!

_
Input Sentence:  I ran.
Deocded Sentence:  אני מתכיר את זה.

_
Input Sentence:  I see.
Deocded Sentence:  אני מרגי